In [ ]:
!python --version
!nvcc --version

Python 3.11.11
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


### Install Dependencies

- `aisuite`: open-source library for ease-of-access to all LLMs
- Currently available:
  -

In [ ]:
!pip install docstring-parser pydantic instructor groq fuzzywuzzy python-Levenshtein --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.7 MB/s eta 0:00:00


### Import dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys

from IPython.display import Markdown as md

# I. Inspect Sample Dataset

Mount your Google drive so that Colab can access the file directory system.

**Note:** Add the entire `UP DSSoc X Vote Report PH` folder as a shortcut to your "My Drive" directory first.

In [ ]:
# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive, userdata

    hf_token = userdata.get('huggingface')
    groq_token = userdata.get('groq')

    gdrive_string = '/content/drive'
    drive.mount(gdrive_string)
    VRPH_fdir = f'{gdrive_string}/MyDrive/UP DSSoc X Vote Report PH'
else:
    from dotenv import load_dotenv
    from huggingface_hub import login

    load_dotenv(dotenv_path='Data/env/.env')

    hf_token = os.getenv("HF_TOKEN")
    groq_token = os.getenv("GROQ_TOKEN")

    if hf_token is None:
        raise ValueError("HF_TOKEN not found in environment variables.")

    login(token=hf_token)
    VRPH_fdir = '.'

Mounted at /content/drive


Load the sample dataset then preview the available columns.

In [ ]:
file_path = f'{VRPH_fdir}/Data/Input/May/May 1/JAN 1 2022-MAY 1 2022 TWEETS.xlsx'
may1_df = pd.read_excel(file_path, sheet_name='Data')
may1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  62 non-null     object
 1   full_text   62 non-null     object
 2   id          62 non-null     int64 
 3   url         62 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.1+ KB


In [ ]:
may1_df.head()

,created_at,full_text,id,url
0,2022-03-31T22:35:10.000Z,ANO TO BATAAN? YUNG SAKIN SIMPLENG PLACARD LAN...,1509660602389524521,https://twitter.com/NongNiko/status/1509660602...
1,2022-02-17T08:42:21.000Z,@lilyoftheveil17 @jabjimenez @COMELEC @DOHgovp...,1494230727025905667,https://twitter.com/reinteee/status/1494230727...
2,2022-03-18T00:54:22.000Z,Hindi ba bawal yan @VoteReportPH #VRPH2022,1504622202154778626,https://twitter.com/KimTomCan/status/150462220...
3,2022-03-08T01:55:22.000Z,Here at the Guiguinto Municipal Oval Uniteam r...,1501013676585021441,https://twitter.com/lianbuan/status/1501013676...
4,2022-02-01T10:51:34.000Z,"Hello, we are VoteReportPH ☀️ a grassroots-ba...",1488465040525053952,https://twitter.com/VoteReportPH/status/148846...


In [ ]:
sample_text = may1_df['full_text'].iloc[may1_df['full_text'].str.len().idxmax()]
md(sample_text)

ANO TO BATAAN? YUNG SAKIN SIMPLENG PLACARD LANG PINASIKAT NYO NA? TAS ITO NANLABAG TALAGA SA BATAS WALANG AKSYON. AYUS!

REPUBLIC ACT NO. 8491

AN ACT PRESCRIBING THE CODE OF THE NATIONAL FLAG, ANTHEM, MOTTO, COAT-OF-ARMS AND OTHER HERALDIC ITEMS AND DEVICES OF THE PHILIPPINES
++ https://t.co/J9RkZQkKdV

# II. Define Groq Client

In [ ]:
from groq import Groq   # LLM inference server (free with limits)
import instructor

## A. Define Client

In [ ]:
client = instructor.from_groq(
    Groq(api_key=groq_token),
    mode=instructor.Mode.JSON
)

## B. Define System Prompt

In [ ]:
system_prompt = """
You are a social media analyst fluent in both English and Philippine languages not just Filipino.
Having an extensive background in the local culture and election-related laws of the Philippines, you are familiar with Republic Act of 9006, known as AN ACT TO ENHANCE THE HOLDING OF FREE, ORDERLY, HONEST, PEACEFUL AND CREDIBLE ELECTIONS THROUGH FAIR ELECTION PRACTICE.
Tasked in monitoring election-related information, your task is to analyze tweets and extract features as requested.
"""
md(system_prompt)


You are a social media analyst fluent in both English and Philippine languages not just Filipino.
Having an extensive background in the local culture and election-related laws of the Philippines, you are familiar with Republic Act of 9006, known as AN ACT TO ENHANCE THE HOLDING OF FREE, ORDERLY, HONEST, PEACEFUL AND CREDIBLE ELECTIONS THROUGH FAIR ELECTION PRACTICE.
Tasked in monitoring election-related information, your task is to analyze tweets and extract features as requested.


# III. Explore Pydantic

In [ ]:
from pydantic import BaseModel  # define schema for LLM outputs

## A. Sample usage with Groq

In [ ]:
class ExtractUser(BaseModel):
    name: str
    age: int

In [ ]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    response_model=ExtractUser,
    messages=[{"role": "user", "content": "Extract Jason is 25 years old."}],
)

print(f"{response.name} is {response.age} years old.")

Jason is 25 years old.


## B. Define Enumerations

In [ ]:
from datetime import datetime
from enum import Enum
import json
from pydantic import Field

In [ ]:
class Violation(Enum):
    IMPROPER_LABELING = "Improper labeling of campaign materials"
    MEDIA_PORTRAYAL = "Portrayal of candidate in movie, documentary, concert etc.",
    MATERIAL_TOO_BIG = "Campaign materials in excess of the size, duration or frequency"
    FALSE_PROPAGANDA = "False/misleading online campaign propaganda"
    BLACK_PROPAGANDA = "Black Propaganda"
    OFFENSIVE_MATERIAL = "Offensive Campaign Material"
    UNAUTHORIZED_DISPLAY = "Campaign Materials Displayed in Unauthorized Areas"
    NONCOMPLIANCE_WITH_HEALTH = "Non-compliance with Health Protocols"
    CAMPAIGNING_OUTSIDE_ALLOWED_PERIOD = "Campaigning during prohibited period"
    OTHER = "Information not available"

class Fraud(Enum):
    PCOS_ERROR = "PCOS Errors"
    REJECTED_BALLOTS = "Rejected ballots"
    BEI_PROCEDURES = "BEI Procedures"
    ELECTION_PARAPHERNALIA = "Election Paraphernalia"
    VOTER_HARASSMENT = "Harassment of voters"
    TRANSMISSION_PROBLEMS = "Transmission problems"
    DISENFRANCHISEMENT = "Disenfranchisement"
    ELECTIONEERING = "Electioneering"
    POWER_INTERRUPTION = "Power interruption"
    VOTERS_LIST = "Voters list"
    TRADITIONAL_FRAUD = "Traditional fraud"
    FTS_PCOS_PROBLEMS = "FTS – PCOS problems"
    VOTE_BUYING = "Vote-buying"
    ELECTION_VIOLENCE = "Election violence"
    ELECTION_PROCESS = "Election process"
    DELAYED_VOTING = "Delayed voting"
    FTS_LACK_OF_MATERIALS = "FTS – lack of materials"
    MANUAL_COUNTING = "Manual counting"
    FTS_BEI_PROCEDURES = "FTS – BEI Procedures"
    CANVASSING_PROBLEMS = "Canvassing Problems"
    CLUSTERED_PRECINCTS = "Clustered Precincts"
    CF_CARDS = "CF Cards"
    BLACK_PROPAGANDA = "Black propaganda"
    FAILURE_OF_ELECTION = "Failure of election"
    ELECTION_RETURNS = "Election returns"
    FTS_OTHERS = "FTS – Others"
    FTS_MILITARY_PRESENCE = "FTS – Military presence"
    FTS_POWER_OUTAGE = "FTS – power outage"
    PRE_SHADED_BALLOTS = "Pre-shaded ballots"
    OTHER = "Information not available"

class ExtractInfo(BaseModel):
    topic: str = Field(description="The main topic of the text.")
    tone: str = Field(description="Emotional tone or context of the text.")
    spam: bool = Field(description="Whether the text is likely spam.")
    location: str = Field(description="Physical location or place the text refers to.")
    date: datetime = Field(description="Date associated with the text.")
    campaign_violations: Violation = Field(description="Any campaign-related violations highlighted in the text.")
    observed_fraud: Fraud = Field(description="Any observed fraudulent activity in the text.")

    def __str__(self) -> str:
        return json.dumps(self.model_dump(), default=str, indent=4)

In [ ]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    response_model=ExtractInfo,
    messages=[
        #{"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Extract: {sample_text}"},
    ],
)
print(response)

{
    "topic": "Election Campaign Violation",
    "tone": "Angry",
    "spam": false,
    "location": "Bataan",
    "date": "2022-01-01 00:00:00",
    "campaign_violations": "Violation.MATERIAL_TOO_BIG",
    "observed_fraud": "Fraud.OTHER"
}


In [ ]:
sample_text

'ANO TO BATAAN? YUNG SAKIN SIMPLENG PLACARD LANG PINASIKAT NYO NA? TAS ITO NANLABAG TALAGA SA BATAS WALANG AKSYON. AYUS!\n\nREPUBLIC ACT NO. 8491\n\nAN ACT PRESCRIBING THE CODE OF THE NATIONAL FLAG, ANTHEM, MOTTO, COAT-OF-ARMS AND OTHER HERALDIC ITEMS AND DEVICES OF THE PHILIPPINES\n++ https://t.co/J9RkZQkKdV'

## C. Robust Categorization

In [ ]:
from typing import Annotated, Literal, Union
from fuzzywuzzy import process as fuzzy_process
from pydantic import BeforeValidator

In [ ]:
class ViolationType(str, Enum):
    IMPROPER_LABELING = "Improper labeling of campaign materials"
    MEDIA_PORTRAYAL = "Portrayal of candidate in movie, documentary, concert etc.",
    MATERIAL_TOO_BIG = "Campaign materials in excess of the size, duration or frequency"
    FALSE_PROPAGANDA = "False/misleading online campaign propaganda"
    BLACK_PROPAGANDA = "Black Propaganda"
    OFFENSIVE_MATERIAL = "Offensive Campaign Material"
    UNAUTHORIZED_DISPLAY = "Campaign Materials Displayed in Unauthorized Areas"
    NONCOMPLIANCE_WITH_HEALTH = "Non-compliance with Health Protocols"
    CAMPAIGNING_OUTSIDE_ALLOWED_PERIOD = "Campaigning during prohibited period"
    OTHER = "Information not available"

class FraudType(str, Enum):
    PCOS_ERROR = "PCOS Errors"
    REJECTED_BALLOTS = "Rejected ballots"
    BEI_PROCEDURES = "BEI Procedures"
    ELECTION_PARAPHERNALIA = "Election Paraphernalia"
    VOTER_HARASSMENT = "Harassment of voters"
    TRANSMISSION_PROBLEMS = "Transmission problems"
    DISENFRANCHISEMENT = "Disenfranchisement"
    ELECTIONEERING = "Electioneering"
    POWER_INTERRUPTION = "Power interruption"
    VOTERS_LIST = "Voters list"
    TRADITIONAL_FRAUD = "Traditional fraud"
    FTS_PCOS_PROBLEMS = "FTS – PCOS problems"
    VOTE_BUYING = "Vote-buying"
    ELECTION_VIOLENCE = "Election violence"
    ELECTION_PROCESS = "Election process"
    DELAYED_VOTING = "Delayed voting"
    FTS_LACK_OF_MATERIALS = "FTS – lack of materials"
    MANUAL_COUNTING = "Manual counting"
    FTS_BEI_PROCEDURES = "FTS – BEI Procedures"
    CANVASSING_PROBLEMS = "Canvassing Problems"
    CLUSTERED_PRECINCTS = "Clustered Precincts"
    CF_CARDS = "CF Cards"
    BLACK_PROPAGANDA = "Black propaganda"
    FAILURE_OF_ELECTION = "Failure of election"
    ELECTION_RETURNS = "Election returns"
    FTS_OTHERS = "FTS – Others"
    FTS_MILITARY_PRESENCE = "FTS – Military presence"
    FTS_POWER_OUTAGE = "FTS – power outage"
    PRE_SHADED_BALLOTS = "Pre-shaded ballots"
    OTHER = "Information not available"

class Violation(BaseModel):
    def convert_str_to_violation_name(v: str | ViolationType) -> ViolationType:
        if isinstance(v, ViolationType):
            return v
        else:
            try:
                match, score = fuzzy_process.extractOne(v.upper(), [e.value for e in list(ViolationType)])
                return ViolationType(match) if score >= 60 else ViolationType.OTHER
            except ValueError:
                return ViolationType.OTHER

    violation_type: Annotated[str, BeforeValidator(convert_str_to_violation_name)]
    violation_mention: str = Field(description="A recognized instance of a campaign violation.")

class Fraud(BaseModel):
    def convert_str_to_fraud_name(v: str | FraudType) -> FraudType:
        if isinstance(v, FraudType):
            return v
        else:
            try:
                match, score = fuzzy_process.extractOne(v.upper(), [e.value for e in list(FraudType)])
                return FraudType(match) if score >= 60 else FraudType.OTHER
            except ValueError:
                return FraudType.OTHER

    fraud_type: Annotated[str, BeforeValidator(convert_str_to_fraud_name)]
    fraud_mention: str = Field(description="A recognized instance of campaign-related fraudulent activity.")

class ExtractInfo(BaseModel):
    topic: str = Field(description="The main topic of the text.")
    tone: str = Field(description="Emotional tone or context of the text.")
    spam: bool = Field(description="Whether the text is likely spam.")
    location: Union[str, Literal["Information not available"]] = Field(description="Physical location or place the text refers to.")
    date: Union[datetime, Literal["Information not available"]] = Field(description="Any date that is in the text")
    campaign_violations: list[Violation] = Field(description="Any campaign-related violations, according to Republic Act. 9006, highlighted in the text.")
    observed_fraud: list[Fraud] = Field(description="Any observed campaign-related fraudulent activity in the text.")

    def __str__(self) -> str:
        return json.dumps(self.model_dump(), default=str, indent=4)

In [ ]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    response_model=ExtractInfo,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Extract: {sample_text}"},
    ],
    temperature=0
)

print(response)

{
    "topic": "Election Campaign Violation",
    "tone": "Angry",
    "spam": false,
    "location": "Bataan",
    "date": "Information not available",
    "campaign_violations": [
        {
            "violation_type": "False/misleading online campaign propaganda",
            "violation_mention": "Pinasikat ng placard"
        }
    ],
    "observed_fraud": []
}


# IV. Create Function to Extract Tweet Features as JSON/Dictionary


## A. `VRPH_Tweet_FeatureExtractor` for single tweets

In [ ]:
def VRPH_Tweet_FeatureExtractor(
    client,
    model,
    response_model,
    system_prompt,
    tweet_text,
    temperature=0
):
  """Extracts features from a tweet using a Large Language Model (LLM).

  This function sends a tweet text to an LLM for analysis,
  requesting it to extract relevant features based on a predefined schema.
  It utilizes the provided client, model, and system prompt for interaction
  with the LLM.

  Args:
      client: The LLM client object (e.g., from Groq or instructor library).
      model: The name of the LLM model to use (e.g., "llama3-70b-8192").
      response_model: The Pydantic model defining the expected output structure.
      system_prompt: The overall instructions for the LLM (e.g., persona, task).
      tweet_text: The text content of the tweet to analyze.

  Returns:
      An instance of the `response_model` containing the extracted features.
      This could be a Pydantic model like `ExtractInfo` with fields like topic,
      tone, spam, location, date, campaign violations, and observed fraud.
  """
  response = client.chat.completions.create(
      model=model,
      response_model=response_model,
      messages=[
          {
              "role": "system",
              "content": system_prompt
              },
          {
              "role": "user",
              "content": f"Extract: {tweet_text}"
              },
          ],
      temperature=temperature,
      )

  return response

In [ ]:
client = instructor.from_groq(
    Groq(api_key=groq_token),
    mode=instructor.Mode.JSON
)

tweet_features_json = VRPH_Tweet_FeatureExtractor(
    client=client,
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    response_model=ExtractInfo,
    system_prompt=system_prompt,
    tweet_text=sample_text
)

print(f"Input Tweet:\n {sample_text}\n\n")
print(f"Extracted Features:\n {tweet_features_json}")

Input Tweet:
 ANO TO BATAAN? YUNG SAKIN SIMPLENG PLACARD LANG PINASIKAT NYO NA? TAS ITO NANLABAG TALAGA SA BATAS WALANG AKSYON. AYUS!

REPUBLIC ACT NO. 8491

AN ACT PRESCRIBING THE CODE OF THE NATIONAL FLAG, ANTHEM, MOTTO, COAT-OF-ARMS AND OTHER HERALDIC ITEMS AND DEVICES OF THE PHILIPPINES
++ https://t.co/J9RkZQkKdV


Extracted Features:
 {
    "topic": "Election-related issue",
    "tone": "Angry",
    "spam": false,
    "location": "Bataan",
    "date": "Information not available",
    "campaign_violations": [
        {
            "violation_type": "Improper labeling of campaign materials",
            "violation_mention": "Pinasikat nyo na ang placard"
        },
        {
            "violation_type": "Information not available",
            "violation_mention": "Walang aksyon sa batas"
        }
    ],
    "observed_fraud": []
}


## B. Apply on DataFrame of May 1 Elections

In [ ]:
import os
import time
import json
import pandas as pd
from tqdm.autonotebook import tqdm
from typing import Callable, Type

In [ ]:
class Tweet_FeatureExtraction_Pipeline:
    def __init__(
        self,
        extractor_fn: Callable[..., any],
        client,
        model: str,
        response_model: Type[BaseModel],
        system_prompt: str,
        tweet_column: str = "tweet_text",
        sleep_time: float = 1.0,
    ):
        """
        Args:
            extractor_fn: function to call for each tweet (e.g. VRPH_Tweet_FeatureExtractor)
            client: LLM client (e.g. instructor.from_groq(...))
            model: name of the LLM model
            response_model: Pydantic model class (e.g. ExtractInfo)
            system_prompt: instructions for the LLM
            tweet_column: name of the column in your DF containing the tweet text
            sleep_time: seconds to sleep between requests
        """
        self.extractor_fn   = extractor_fn
        self.client         = client
        self.model          = model
        self.response_model = response_model
        self.system_prompt  = system_prompt
        self.tweet_column   = tweet_column
        self.sleep_time     = sleep_time

        # derive output fields from the Pydantic model, plus an "error" column
        self.new_columns = list(self.response_model.model_fields.keys()) + ["error"]

    def feature_extract(
        self,
        df: pd.DataFrame,
        output_csv: str,
        resume: bool = True
    ) -> pd.DataFrame:
        """
        Runs feature extraction on every row of df and writes to output_csv.

        If resume=True and output_csv exists, loads it and continues at the first
        row where the first new_column is null.
        """
        # — prepare output DataFrame
        if resume and os.path.exists(output_csv):
            out_df = pd.read_csv(output_csv)
        else:
            out_df = df.copy()
            for col in self.new_columns:
                out_df[col] = None

        # — find where to start (first null in the first response field)
        first_field = self.new_columns[0]
        null_mask   = out_df[first_field].isna()
        if null_mask.any():
            start_idx = int(null_mask.idxmax())
        else:
            print("All rows already processed.")
            return out_df

        # — iterate with tqdm
        for idx in tqdm(range(start_idx, len(out_df)), desc="Extracting tweet features", unit='tweet'):
            text = out_df.at[idx, self.tweet_column]
            try:
                resp = self.extractor_fn(
                    client=self.client,
                    model=self.model,
                    response_model=self.response_model,
                    system_prompt=self.system_prompt,
                    tweet_text=text,
                    temperature=0
                )
                # normalize to dict
                if hasattr(resp, "model_dump"):
                    data = resp.model_dump()
                else:
                    data = resp if isinstance(resp, dict) else json.loads(resp.json())

                # fill each field dynamically
                for field in self.response_model.model_fields:
                    value = data.get(field)
                    # for list-of-dicts, store as JSON string
                    if isinstance(value, (list, dict)):
                        out_df.at[idx, field] = json.dumps(value)
                    else:
                        out_df.at[idx, field] = value

                out_df.at[idx, "error"] = None

            except Exception as e:
                out_df.at[idx, "error"] = str(e)

            # save progress after each row
            out_df.to_csv(output_csv, index=False)
            time.sleep(self.sleep_time)

        return out_df


In [ ]:
%%time
pipeline = Tweet_FeatureExtraction_Pipeline(
    extractor_fn=VRPH_Tweet_FeatureExtractor,
    client=client,
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    response_model=ExtractInfo,
    system_prompt=system_prompt,
    tweet_column="full_text",
    sleep_time=4.0,
)

output_df = pipeline.feature_extract(
    df=may1_df,
    output_csv=f'{VRPH_fdir}/Data/Output/2022_05_01_TweetFeatures.csv',
    resume=True
)
output_df.info()

Extracting tweet features:   0%|          | 0/62 [00:00<?, ?tweet/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   created_at           62 non-null     object
 1   full_text            62 non-null     object
 2   id                   62 non-null     int64 
 3   url                  62 non-null     object
 4   topic                62 non-null     object
 5   tone                 62 non-null     object
 6   spam                 62 non-null     object
 7   location             62 non-null     object
 8   date                 62 non-null     object
 9   campaign_violations  62 non-null     object
 10  observed_fraud       62 non-null     object
 11  error                0 non-null      object
dtypes: int64(1), object(11)
memory usage: 5.9+ KB
CPU times: user 2.99 s, sys: 345 ms, total: 3.34 s
Wall time: 6min 52s


In [ ]:
output_df['tone'].value_counts()

tone
Neutral                17
Informative             5
Concerned               4
Inquiry                 3
neutral                 3
informative             3
Critical                3
Warning                 2
Call to Action          2
Angry                   1
critical                1
neutral/informative     1
Concerned/Urgent        1
Cautionary              1
Suspicious              1
Neutral/Inquiry         1
Neutral/Informative     1
Informative/Urgent      1
Urgent/Alarm            1
excited                 1
Neutral/Inquiring       1
defensive               1
Anger/Frustration       1
Positive                1
Outrage                 1
Neutral/Positive        1
encouragement           1
inquiry                 1
Neutral/Humorous        1
Name: count, dtype: int64